In [ ]:
from sentence_transformers import SentenceTransformer
import chromadb
import openai
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = api_key

# ChromaDB 및 모델 초기화
model = SentenceTransformer('all-MiniLM-L6-v2')
client = chromadb.PersistentClient()


# 기존 컬렉션 로드 또는 생성
if "crawled_data" not in [col.name for col in client.list_collections()]:
    collection = client.create_collection(name="crawled_data")
else:
    collection = client.get_collection(name="crawled_data")

# 쿼리 함수 정의
def query_rag(query_text):
    # 쿼리 임베딩 생성
    query_embedding = model.encode([query_text])[0]
    
    # 벡터 검색
    search_results = collection.query(
        query_embeddings=[query_embedding],
        n_results=4,
        include=["documents"]
    )
    
     # 검색된 문서 리스트 가져오기
    retrieved_docs = search_results['documents'][0] if search_results['documents'] else []
    if not retrieved_docs:
        return "관련 문서를 찾을 수 없습니다."
    
    # LLM을 통한 최종 응답 생성
    context = "\n".join(retrieved_docs)  # 문서들 연결
    response = openai.chat.completions.create(
        messages=[
            {"role": "system", "content": "당신은 문서 정보를 기반으로 주로 자립준비청년들의 질문에 답변하는 어시스턴트입니다."},
            {"role": "user", "content": f"다음 정보들을 바탕으로 '{query_text}'에 대한 응답을 생성하세요:\n\n{context}\n\n답변:"}
        ],
        model="gpt-4o",  # 최신 모델 지정
    )
    
    return response.choices[0].message.content;

# 예시 쿼리
result = query_rag("난 집에만 있는데... 나도 다시 사람들을 만나고 싶어. 뭐 좋은 프로그램이 없을까?")
print(result)

### 만약 실행 안되시면  embedding.ipynb파일에 복붙해서 함 실행해보세요!!